In [137]:
from selenium.webdriver.chrome.options import Options 
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import random
import csv
import pandas as pd
import re
import datetime

In [138]:
url_base = "https://www.themoviedb.org"

options = Options()
options.add_argument("--headless")

driver = webdriver.Chrome("/usr/local/bin/chromedriver", options=options)
driver.get(url_base+"/movie")

In [139]:
element = driver.find_element_by_partial_link_text("Load") 
driver.execute_script("arguments[0].click();", element)
time.sleep(1)

url_single_movies = []
max_iterations = 0
iter=1
while max_iterations < 5:
    scrollHeight = driver.execute_script("return document.documentElement.scrollHeight")
    #print("scrollHeight: "+str(scrollHeight))
    Height=4200*iter
    #print("Height: "+str(Height))
    driver.execute_script("window.scrollTo(0, " + str(Height) + ");")
    if Height > scrollHeight:
        print('End of page')
        break
    time.sleep(random.randint(1, 3))
    max_iterations+=1
    #print("Max iterations: "+str(max_iterations))
    iter+=1
    #print("iter: "+str(iter))

body = driver.execute_script("return document.body")
source = body.get_attribute('innerHTML') 

soup = BeautifulSoup(source, "html.parser")
movies = soup.find_all("div", {"class": "card style_1"})

print(len(movies))

120


In [140]:
for movie in movies:
    title = movie.h2.a["href"] 
    url_single_movies.append(title)

print(url_single_movies)

['/movie/527774', '/movie/793723', '/movie/587996', '/movie/587807', '/movie/458576', '/movie/464052', '/movie/791373', '/movie/484718', '/movie/581389', '/movie/522444', '/movie/775996', '/movie/602269', '/movie/649087', '/movie/613504', '/movie/651571', '/movie/590706', '/movie/755812', '/movie/560144', '/movie/630586', '/movie/604822', '/movie/581387', '/movie/399566', '/movie/529203', '/movie/495764', '/movie/664767', '/movie/607383', '/movie/508442', '/movie/595813', '/movie/539885', '/movie/553604', '/movie/577922', '/movie/786300', '/movie/682254', '/movie/544401', '/movie/38700', '/movie/596247', '/movie/520946', '/movie/141052', '/movie/475557', '/movie/599281', '/movie/621876', '/movie/373571', '/movie/638597', '/movie/531499', '/movie/644092', '/movie/631939', '/movie/338762', '/movie/400160', '/movie/299536', '/movie/384018', '/movie/737568', '/movie/666750', '/movie/615677', '/movie/531219', '/movie/283566', '/movie/545609', '/movie/621013', '/movie/582306', '/movie/461130

In [141]:
data_file = open('dataset_peliculas.csv', 'w', newline='', encoding='utf-8') 
csv_writer = csv.writer(data_file, delimiter=";") 
csv_writer.writerow(["Título","Título original","Idioma original","Duración","Directores","Sinopsis","Año de estreno","Clasificación por edad","Imagen cartel","Puntuación"])

132

In [142]:
for url_movie in url_single_movies:
    driver.get(url_base+url_movie)
    body = driver.execute_script("return document.body")
    source = body.get_attribute('innerHTML') 
    soup = BeautifulSoup(source, "html.parser")

    poster_wrapper = soup.find("div", {"class": "poster_wrapper"})
    header_wrapper = soup.find("div", {"class": "header_poster_wrapper"})
    column_wrapper = soup.find("div", {"class": "grey_column"})

    
    titulo = header_wrapper.find("h2").a.contents[0]
    
    try:
        idioma_original = re.sub('Original Language ', '',column_wrapper.find("section",{"class":"facts left_column"}).find_all("p")[2].text)
        tit_original = re.sub('Original Title ','', column_wrapper.find("p",{"class":"wrap"}).text)
    except AttributeError:
        tit_original = titulo
        idioma_original = re.sub('Original Language ', '',column_wrapper.find("section",{"class":"facts left_column"}).find_all("p")[1].text)
    
    try:
        duracion = header_wrapper.find("span", {"class":"runtime"}).text.strip()
    except AttributeError:
        duracion = "No disponible"
    
    directores = header_wrapper.find("li",{"class":"profile"}).p.text
    
    sinopsi = header_wrapper.find_all("div",{"class": "overview"})[0].text.strip()
    
    year = datetime.datetime.strptime(header_wrapper.find("span", {"class": "release"}).contents[0].strip()[:10], "%m/%d/%Y").year
    
    genres = [header_wrapper.find("span", {"class":"genres"}).find_all("a")[genre].text 
              for genre in range(len(header_wrapper.find("span", {"class":"genres"}).find_all("a")))]
    genres = ", ".join(genres)

    imagen = poster_wrapper.find("img").attrs['src']
    puntuacion = list(header_wrapper.find("span",{"class":"icon"}).attrs.values())[0][1][-2:]
    
    movie =  [
        titulo, tit_original, idioma_original, duracion, directores,
        sinopsi, year, genres, imagen, puntuacion
    ]

    csv_writer.writerow(movie)
    time.sleep(random.randint(1, 3))

In [143]:
data_file.close()
driver.close()
df_movies = pd.read_csv("dataset_peliculas.csv", sep=";", encoding="utf-8")
df_movies

,Título,Título original,Idioma original,Duración,Directores,Sinopsis,Año de estreno,Clasificación por edad,Imagen cartel,Puntuación
0,Raya and the Last Dragon,Raya and the Last Dragon,English,1h 47m,Carlos López Estrada,"Long ago, in the fantasy world of Kumandra, hu...",2021,"Animation, Adventure, Fantasy, Family, Action",/t/p/w300_and_h450_bestv2/lPsD10PP4rgUGiGR4CCX...,85
1,Sentinelle,Sentinelle,French,1h 20m,Julien Leclercq,Transferred home after a traumatizing combat m...,2021,"Thriller, Action, Drama",/t/p/w300_and_h450_bestv2/fFRq98cW9lTo6di2o4lK...,61
2,Below Zero,Bajocero,Spanish; Castilian,1h 46m,Lluís Quílez,"When a prisoner transfer van is attacked, the ...",2021,"Action, Crime, Thriller",/t/p/w300_and_h450_bestv2/dWSnsAGTfc8U27bWsy2R...,64
3,Tom & Jerry,Tom & Jerry,English,1h 41m,William Hanna,Cinephiles Alley Return,2021,"Action, Comedy, Family, Animation",/t/p/w300_and_h450_bestv2/6KErczPBROQty7QoIsaa...,75
4,Monster Hunter,Monster Hunter,English,1h 44m,Paul W. S. Anderson,A portal transports Cpt. Artemis and an elite ...,2020,"Fantasy, Action, Adventure",/t/p/w300_and_h450_bestv2/1UCOF11QCw8kcqvce8LK...,72
...,...,...,...,...,...,...,...,...,...,...
115,Coming to America,Coming to America,English,1h 57m,John Landis,An African prince decides it’s time for him to...,1988,"Comedy, Romance",/t/p/w300_and_h450_bestv2/8YZiA1o264dk0cr1USyM...,68
116,The Rhythm Section,The Rhythm Section,English,1h 49m,Mark Burnell,After the death of her family in an airplane c...,2020,"Action, Thriller",/t/p/w300_and_h450_bestv2/spTr0VYyRtl36Lkk6nCn...,56
117,The Meg,The Meg,English,1h 53m,Jon Turteltaub,A deep sea submersible pilot revisits his past...,2018,"Action, Science Fiction, Horror",/t/p/w300_and_h450_bestv2/xqECHNvzbDL5I3iiOVUk...,62
118,Time to Hunt,사냥의 시간,Korean,2h 15m,Yoon Sung-hyun,Four young men who want to leave their dystopi...,2020,"Drama, Action, Thriller",/t/p/w300_and_h450_bestv2/bkuuvDoPkOJpg0ZDzHkU...,73
